In [21]:
def create_rotated_rectangle(center, width, height, angle, crs="EPSG:4326"):
    """
    Create a rotated rectangle as a polygon in geographic coordinates.
    
    Parameters:
        center (tuple): The (longitude, latitude) coordinates of the centroid.
        width (float): The width of the rectangle.
        height (float): The height of the rectangle.
        angle (float): The rotation angle in degrees (counterclockwise).
        crs (str): Coordinate reference system for geographic coordinates (default: EPSG:4326).
    
    Returns:
        dict: GeoJSON-like structure containing the rectangle coordinates in longitude and latitude.
    """
    # Create rectangle corners centered at (0, 0) in a planar space
    rectangle = Polygon([
        (-width / 2, -height / 2),
        (width / 2, -height / 2),
        (width / 2, height / 2),
        (-width / 2, height / 2),
        (-width / 2, -height / 2)
    ])
    
    # Rotate the rectangle
    rotated_rectangle = rotate(rectangle, angle, origin=(0, 0), use_radians=False)
    
    # Translate the rectangle to the given center
    translated_rectangle = translate(rotated_rectangle, xoff=center[0], yoff=center[1])
    
    # Convert planar coordinates to geographic coordinates (if needed)
    transformer = Transformer.from_crs(crs, crs, always_xy=True)  # Ensure geographic CRS
    geographic_coords = [
        transformer.transform(x, y) for x, y in translated_rectangle.exterior.coords
    ]
    
    # Return as GeoJSON-like dictionary
    return {
        "type": "Polygon",
        "coordinates": [geographic_coords]
    }
    
# Convert the 'geometry' column to MULTIPOLYGON
def convert_to_multipolygon(geojson_dict):
    # Convert GeoJSON-like dict to Shapely geometry
    polygon = shape(geojson_dict)
    
    # Ensure the geometry is a MULTIPOLYGON
    if polygon.geom_type == "Polygon":
        return MultiPolygon([polygon])
    return polygon

In [22]:
# Load your GeoJSON file with points (Replace 'points.geojson' with your file path)
gdf = gpd.read_file(geojson_file)

# Set your desired width, height, and rotation angle (in degrees)
width = 0.00008  # Specify width of the rectangle
height = 0.000009  # Specify height of the rectangle
rotation = 6  # Specify rotation angle in degrees

# Create a new column for the generated polygons
gdf['polygons'] = gdf.geometry.apply(lambda point: create_rotated_rectangle((point.x, point.y), width, height, rotation))
gdf.drop(columns=["geometry"], inplace=True)
gdf.rename(columns={"polygons": "geometry"}, inplace=True)

# Apply conversion of geometry format
gdf['geometry'] = gdf['geometry'].apply(convert_to_multipolygon)

# Ensure your GeoDataFrame has a CRS
gdf.set_crs("EPSG:4326", inplace=True)  # WGS84 CRS
# Save the resulting GeoDataFrame as a new GeoJSON file
gdf.to_file(output_file_skewed, driver='GeoJSON')

DataSourceError: oats_points.geojson: No such file or directory

# Shrink the sowing polygons (without rotating them) to get the extraction polygons

In [ ]:
import geopandas as gpd
from shapely.affinity import scale

def shrink_geometry(geometry, x_factor, y_factor):
    """
    Shrink a polygon's geometry by independent x and y scaling factors.

    Parameters:
        geometry (shapely.geometry): The polygon geometry to shrink.
        x_factor (float): Scaling factor for the x-axis.
        y_factor (float): Scaling factor for the y-axis.

    Returns:
        shapely.geometry: The scaled polygon geometry.
    """
    centroid = geometry.centroid  # Get the centroid of the polygon
    # Scale the polygon relative to its centroid
    return scale(geometry, xfact=x_factor, yfact=y_factor, origin=(centroid.x, centroid.y))

def shrink_polygons(input_file, x_factor=0.9, y_factor=0.9):
    """
    Shrink polygons in a GeoJSON file by independent x and y scaling factors.

    Parameters:
        input_file (str): Path to the input GeoJSON file.
        x_factor (float): Scaling factor for the x-axis (e.g., 0.9 for 90% size).
        y_factor (float): Scaling factor for the y-axis (e.g., 0.9 for 90% size).

    Returns:
        None
    """
    # Load the GeoJSON as a GeoDataFrame
    gdf = input_file

    # Apply scaling to each polygon in the GeoDataFrame
    gdf['geometry'] = gdf['geometry'].apply(lambda geom: shrink_geometry(geom, x_factor, y_factor))

    # Save the updated GeoDataFrame to a new GeoJSON file
    return gdf

# # Example usage
# input_geojson = gpd.read_file("gene2bread.geojson")
# output_geojson = "shrunk_polygons.geojson"

# # Load your GeoJSON file with points

# shrunk_gdf = shrink_polygons(input_geojson, x_factor=0.8, y_factor=0.7)
# shrunk_gdf
# shrunk_gdf.to_file(output_geojson, driver="GeoJSON")  # Save updated GeoDataFrame

# Shrink and rotate the sowing polygons to get the extraction polygons. Corrected where it was skeweing if the x and y factor were not the same

In [8]:
from shapely.affinity import rotate, scale
from shapely.geometry import shape
from pyproj import Transformer, CRS
from shapely.ops import transform
import geopandas as gpd

def scale_and_rotate_polygons(gdf, x_factor, y_factor, angle, input_crs="EPSG:4326", output_crs="EPSG:3857"):
    """
    Scale and rotate polygons in a GeoDataFrame.
    
    Parameters:
        gdf (GeoDataFrame): GeoDataFrame with polygons.
        x_factor (float): Scaling factor for the x-axis.
        y_factor (float): Scaling factor for the y-axis.
        angle (float): Rotation angle in degrees (counterclockwise).
        input_crs (str): The original CRS of the data (default: EPSG:4326).
        output_crs (str): The projected CRS for transformations (default: EPSG:3857).
    
    Returns:
        GeoDataFrame: Updated GeoDataFrame with scaled and rotated polygons.
    """
    # Initialize transformers for CRS transformations
    transformer_to_planar = Transformer.from_crs(input_crs, output_crs, always_xy=True)
    transformer_to_geo = Transformer.from_crs(output_crs, input_crs, always_xy=True)
    
    def transform_geometry(geometry):
        # Transform the geometry to a planar CRS
        planar_geom = transform(lambda x, y: transformer_to_planar.transform(x, y), geometry)
        
        # Uniformly scale the geometry in the planar CRS to avoid skew
        uniform_scaled_geom = scale(planar_geom, xfact=(x_factor + y_factor) / 2, yfact=(x_factor + y_factor) / 2, origin='center')
        
        # Rotate the geometry
        rotated_geom = rotate(uniform_scaled_geom, angle, origin='center', use_radians=False)
        
        # Transform the geometry back to the geographic CRS
        geo_geom = transform(lambda x, y: transformer_to_geo.transform(x, y), rotated_geom)
        return geo_geom
    
    # Apply the transformation to each geometry in the GeoDataFrame
    gdf['geometry'] = gdf['geometry'].apply(transform_geometry)
    return gdf


# Example usage

# # Step 1: Read GeoJSON file
# input_file = "gene2bread.geojson"
# output_file = "transform________d_polygons.geojson"
# gdf = gpd.read_file(input_file)

# # Apply scaling and rotation
# scaled_rotated_gdf = scale_and_rotate_polygons(
#     gdf, x_factor=0.9, y_factor=0.8, angle=0.5, input_crs="EPSG:4326", output_crs="EPSG:3857"
# )

# # Save updated GeoDataFrame to GeoJSON
# scaled_rotated_gdf.to_file("scaled_rotated_po lygons.geojson", driver="GeoJSON")

# NOTE: Although shrinking the polygons work but it is tricky to independelty change the x_factor without disturbing the y_facotr and viceversa since the fields and eventually the polygons are not aligned with the coordinates axis. So, the best approach would be to find the centroids of the sowing polygons and create the shrinked polygons aroung those centroids.

# FINAL APPROACH

In [1]:
import os
import math
import glob
import geopandas as gpd
from pyproj import Transformer, CRS
from shapely.affinity import rotate, translate
from shapely.geometry import Point, Polygon, shape, MultiPolygon, MultiPoint

In [2]:
# Function to find the tif files in a given folder¶
def find_files_in_folder(folder_path, extension=None, recursive=False):
    """
    Retrieves a list of file paths in a specified folder, optionally filtered by file extension,
    and optionally including subdirectories.

    Parameters:
    - folder_path (str): The path of the folder to search for files.
    - extension (str, optional): The file extension to filter by (e.g., "txt" or "tif").
                                 If None, the function lists all files.
    - recursive (bool, optional): If True, includes files from all subdirectories within `folder_path`.
                                  Defaults to False (only lists files in the specified folder).

    Returns:
    - list of str: A list of file paths that match the specified extension in the folder.
                   If no matching files are found, returns a list containing an empty string.

    Example:
    >>> find_files_in_folder("/path/to/folder", "tif")
    ['/path/to/folder/file1.tif', '/path/to/folder/file2.tif']

    >>> find_files_in_folder("/path/to/folder", recursive=True)
    ['/path/to/folder/file1.tif', '/path/to/folder/subfolder/file2.txt', '/path/to/folder/file3.jpg']
    """
    
    matched_files = []

    # Determine the search pattern based on whether an extension is provided and recursion is enabled
    if extension:
        if recursive:
            search_pattern = os.path.join(folder_path, f"**/*.{extension}")
        else:
            search_pattern = os.path.join(folder_path, f"*.{extension}")
    else:
        # No extension specified, handle both recursive and non-recursive cases
        if recursive:
            search_pattern = os.path.join(folder_path, "**/*")
        else:
            search_pattern = os.path.join(folder_path, "*")
    
    # Use glob to find matching files in the specified directory and subdirectories if recursive
    matched_files.extend(glob.glob(search_pattern, recursive=recursive))
    
    # If no files are found, return a list with an empty string
    if not matched_files:
        matched_files = [""]

    return matched_files

In [3]:
def polygons_to_centroids(input_geojson, output_geojson):
    """
    Convert a GeoJSON file of polygons to a GeoJSON file of centroid points.
    
    Parameters:
        input_geojson (str): Path to the input GeoJSON file containing polygons.
        output_geojson (str): Path to the output GeoJSON file containing centroid points.
    
    Returns:
        None
    """
    # Load the GeoJSON file as a GeoDataFrame
    gdf = gpd.read_file(input_geojson)
    
    # Calculate centroids for each polygon
    gdf['geometry'] = gdf['geometry'].centroid
    
    # Ensure geometries are now Point objects
    if not all(isinstance(geom, Point) for geom in gdf['geometry']):
        raise ValueError("Not all geometries are centroids (points).")
    
    # Save the GeoDataFrame with centroids as a new GeoJSON file
    gdf.to_file(output_geojson, driver='GeoJSON')



# Reorder the centroids
def reorder_points_by_columns(input_geojson, output_geojson, rotation_angle=0):
    """
    Reorder points in a GeoJSON file such that points are ordered column-first.
    The rotation is used only for ordering purposes; the final output retains original coordinates.
    
    Parameters:
        input_geojson (str): Path to the input GeoJSON file containing points.
        output_geojson (str): Path to the output GeoJSON file with reordered points.
        rotation_angle (float): Angle in degrees to rotate the grid for ordering (default: 0).
    
    Returns:
        None
    """
    # Load GeoJSON file into a GeoDataFrame
    gdf = gpd.read_file(input_geojson)
    
    # Compute the centroid of the whole grid
    grid_centroid = MultiPoint(gdf.geometry).centroid
    
    # Rotate the points temporarily for ordering
    if rotation_angle != 0:
        gdf['rotated_geometry'] = gdf['geometry'].apply(
            lambda geom: rotate(geom, rotation_angle, origin=grid_centroid, use_radians=False)
        )
    else:
        gdf['rotated_geometry'] = gdf['geometry']
    
    # Extract coordinates from rotated geometries for sorting
    gdf['x'] = gdf['rotated_geometry'].apply(lambda geom: geom.x)
    gdf['y'] = gdf['rotated_geometry'].apply(lambda geom: geom.y)
    
    # Sort by x (columns) first, and then by y (rows) within each column
    gdf_sorted = gdf.sort_values(by=['x', 'y'], ascending=[True, False]).reset_index(drop=True)

    # Drop geometry column for exporting the rotated geometry
    gdf_rotated_geometry_sorted = gdf_sorted.drop(columns=['geometry', 'x', 'y'])
    gdf_rotated_geometry_sorted.rename(columns={"rotated_geometry": "geometry"}, inplace=True)

    # Save the rotated GeoDataFrame as a new GeoJSON file
    gdf_rotated_geometry_sorted.to_file(modify_filename(output_geojson, suffix = "_rotated_geometry"), driver="GeoJSON")
    
    # Drop temporary columns for sorting
    gdf_sorted = gdf_sorted.drop(columns=['rotated_geometry', 'x', 'y'])
    
    # Save the reordered GeoDataFrame as a new GeoJSON file
    gdf_sorted.to_file(output_geojson, driver="GeoJSON")

def update_ids(input_geojson, output_centroids_with_IDs, id_start=100, id_end=111, step=100):
    """
    Update the IDs of entries in a GeoDataFrame following a specific sequence.

    Parameters:
        gdf (GeoDataFrame): Input GeoDataFrame.
        id_start (int): Starting value for each sequence.
        id_end (int): Ending value (inclusive) for each sequence.
        step (int): Step to increase the starting value after each sequence.

    Returns:
        GeoDataFrame: Updated GeoDataFrame with modified IDs.
    """

    gdf = gpd.read_file(input_geojson)

    id_list = []
    current_start = id_start

    while len(id_list) < len(gdf):
        id_list.extend(range(current_start, current_start + (id_end - id_start) + 1))
        current_start += step

    # Ensure the ID list matches the length of the GeoDataFrame
    id_list = id_list[:len(gdf)]
    gdf['ID'] = id_list

    gdf.to_file(output_centroids_with_IDs, driver="GeoJSON")  # Save updated GeoDataFrame


# Correct coordinates system

In [4]:
# In case the coordinates of a certain sowing geojson are not in longlat; use this funciton to convert them to longlat

def convert_utm_to_longlat(input_geojson, output_geojson, utm_zone):
    """
    Convert a GeoJSON file with UTM coordinates to Latitude/Longitude (WGS84).
    
    Parameters:
        input_geojson (str): Path to the input GeoJSON file with UTM coordinates.
        output_geojson (str): Path to the output GeoJSON file with WGS84 coordinates.
        utm_zone (int): UTM zone number for the input coordinates (positive for north, negative for south).
        
    Returns:
        None
    """
    # Load the GeoJSON file as a GeoDataFrame
    gdf = gpd.read_file(input_geojson)
    
    # Define the source CRS as the UTM zone (EPSG code)
    if utm_zone > 0:
        source_crs = CRS.from_epsg(32600 + utm_zone)  # Northern hemisphere
    else:
        source_crs = CRS.from_epsg(32700 - utm_zone)  # Southern hemisphere
    
    # Set the GeoDataFrame's CRS
    gdf = gdf.set_crs(source_crs)
    
    # Convert to WGS84 (longitude/latitude)
    gdf = gdf.to_crs("EPSG:4326")
    
    # Save the converted GeoDataFrame to a new GeoJSON file
    gdf.to_file(output_geojson, driver="GeoJSON")

# # Path to the input and output GeoJSON files
# input_geojson = "path_to_utm_file.geojson"
# output_geojson = "path_to_longlat_file.geojson"

# # Specify the UTM zone of the input data (e.g., zone 33N is 33, zone 33S is -33)
# utm_zone = 32

# # Convert the GeoJSON
# convert_utm_to_longlat(input_geojson, output_geojson, utm_zone)

In [5]:
def create_rotated_rectangle(center, width, height, angle, crs="EPSG:4326"):
    """
    Create a rotated rectangle as a polygon in geographic coordinates around the given center point.
    
    Parameters:
        center (tuple): The (longitude, latitude) coordinates of the centroid.
        width (float): The width of the rectangle (in meters or appropriate planar units).
        height (float): The height of the rectangle (in meters or appropriate planar units).
        angle (float): The rotation angle in degrees (counterclockwise).
        crs (str): Coordinate reference system for geographic coordinates (default: EPSG:4326).
    
    Returns:
        dict: GeoJSON-like structure containing the rectangle coordinates in longitude and latitude.
    """
    # Define a planar CRS (e.g., UTM zone based on the center point)
    transformer_to_planar = Transformer.from_crs(crs, CRS("EPSG:3857"), always_xy=True)
    transformer_to_geo = Transformer.from_crs(CRS("EPSG:3857"), crs, always_xy=True)

    # Transform the center to planar coordinates
    center_planar = transformer_to_planar.transform(center[0], center[1])
    
    # Create rectangle corners centered at (0, 0) in the planar space
    rectangle = Polygon([
        (-width / 2, -height / 2),
        (width / 2, -height / 2),
        (width / 2, height / 2),
        (-width / 2, height / 2),
        (-width / 2, -height / 2)
    ])
    
    # Rotate the rectangle
    rotated_rectangle = rotate(rectangle, angle, origin=(0, 0), use_radians=False)
    
    # Translate the rectangle to the given center in planar coordinates
    translated_rectangle = translate(rotated_rectangle, xoff=center_planar[0], yoff=center_planar[1])
    
    # Convert planar coordinates back to geographic coordinates
    geographic_coords = [
        transformer_to_geo.transform(x, y) for x, y in translated_rectangle.exterior.coords
    ]
    
    # Return as GeoJSON-like dictionary
    return {
        "type": "Polygon",
        "coordinates": [geographic_coords]
    }

# Convert the 'geometry' column to MULTIPOLYGON
def convert_to_multipolygon(geojson_dict):
    # Convert GeoJSON-like dict to Shapely geometry
    polygon = shape(geojson_dict)
    
    # Ensure the geometry is a MULTIPOLYGON
    if polygon.geom_type == "Polygon":
        return MultiPolygon([polygon])
    return polygon

In [6]:
def generate_shrinked_polygons(
    input_centroids_with_IDs,
    output_extraction_polygons,
    width,
    height,
    rotation,
    crs="EPSG:4326",
):
    """
    Generate shrinked polygons around centroids, save them to a GeoJSON file, and apply transformations.

    Parameters:
        input_centroids_with_IDs (str): Path to the input GeoJSON file with centroids and IDs.
        output_extraction_polygons (str): Path to the output GeoJSON file for the polygons.
        width (float): Width of the polygons to generate.
        height (float): Height of the polygons to generate.
        rotation (float): Rotation angle of the polygons in degrees.
        crs (str): Coordinate Reference System for the GeoDataFrame (default: "EPSG:4326").

    Returns:
        None
    """

    # Load the GeoJSON file with centroids and IDs
    gdf = gpd.read_file(input_centroids_with_IDs)

    # Create shrinked polygons based on the centroids
    gdf['polygons'] = gdf.geometry.apply(
        lambda point: create_rotated_rectangle(
            (point.x, point.y), width, height, rotation
        )
    )
    
    # Replace the geometry column with the generated polygons
    gdf.drop(columns=["geometry"], inplace=True)
    gdf.rename(columns={"polygons": "geometry"}, inplace=True)
    
    # Convert the polygons to MultiPolygon format
    gdf['geometry'] = gdf['geometry'].apply(convert_to_multipolygon)
    
    # Ensure the GeoDataFrame has a valid CRS
    gdf.set_crs(crs, inplace=True)
    
    # Save the resulting GeoDataFrame as a GeoJSON file
    gdf.to_file(output_extraction_polygons, driver='GeoJSON')

    print(f"Shrinked polygons saved to {output_extraction_polygons}")

In [7]:

def modify_filename(filepath, prefix=None, suffix=None, subfolder=None):
    """
    Modify a file path by adding a prefix, suffix, and optionally creating a new subfolder.

    Parameters:
        filepath (str): The original file path.
        prefix (str, optional): Text to add before the file name. Defaults to None.
        suffix (str, optional): Text to add before the file extension. Defaults to None.
        subfolder (str, optional): Name of a subfolder to include in the path. If provided, the subfolder will be created if it doesn't exist.

    Returns:
        str: The modified file path.
    """
    # Split the file path into directory, filename, and extension
    dir_path, filename = os.path.split(filepath)
    name, ext = os.path.splitext(filename)
    
    # Add the subfolder if provided
    if subfolder:
        dir_path = os.path.join(dir_path, subfolder)
        os.makedirs(dir_path, exist_ok=True)  # Create the subfolder if it doesn't exist
    
    # Add prefix and suffix
    if prefix:
        name = prefix + name
    if suffix:
        name = name + suffix

    # Return the modified file path
    return os.path.join(dir_path, name + ext)

In [8]:
def create_metadata_based_on_filename(filenames, conditions, metadata_keys):
    """
    Create a metadata dictionary for each filename based on the matching conditions.

    Parameters:
        filenames (list): List of filenames to process.
        conditions (dict): Dictionary where keys are substrings to match in filenames,
                           and values are tuples containing metadata values.
        metadata_keys (list): List of keys that define the metadata variables to extract.

    Returns:
        dict: Dictionary where each filename is mapped to its metadata dictionary.
    """
    metadata = {}

    # Iterate over each filename
    for filename in filenames:
        filename_lower = filename.lower()
        metadata_dict = {}

        # Check each condition to see if it matches the filename
        for key, condition in conditions.items():
            if key.lower() in filename_lower:
                # Map the tuple values to the metadata keys
                metadata_dict = dict(zip(metadata_keys, condition))

        # Add the metadata dictionary for the current filename
        metadata[filename] = metadata_dict

    return metadata

# Convert GEOJSON to XLSX and vice versa

In [1]:
import os  # To work with file paths
import json
import pandas as pd

def geojson_to_xlsx(geojson_file):
    # Load GeoJSON data
    with open(geojson_file, 'r') as f:
        geojson_data = json.load(f)
    
    # Extract features
    features = geojson_data['features']
    
    # Create a DataFrame from the features
    df = pd.json_normalize(features)

    xlsx_file = os.path.splitext(geojson_file)[0] + ".xlsx"

    # Save DataFrame to Excel file
    df.to_excel(xlsx_file, index=False)
    print(f"GeoJSON data has been converted to {xlsx_file}")

In [2]:
import os  # To work with file paths
import json
import pandas as pd

def xlsx_to_geojson_beautifued(xlsx_file):
    # Extract the name from the xlsx_file
    name = os.path.splitext(os.path.basename(xlsx_file))[0]
    geojson_file = os.path.splitext(xlsx_file)[0] + ".geojson"
    
    # Load Excel data into a DataFrame
    df = pd.read_excel(xlsx_file)

    # Ensure required columns are present
    required_columns = ['geometry.type', 'geometry.coordinates']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"Missing required column: '{col}' in the Excel file.")

    # Convert each row to a GeoJSON feature
    features = []
    for _, row in df.iterrows():
        geometry = {
            "type": row['geometry.type'],
            "coordinates": json.loads(row['geometry.coordinates'])  # Parse coordinates from string to Python object
        }
        # Collect all properties except the geometry columns
        property_columns = [col for col in df.columns if col not in required_columns]
        properties = row[property_columns].to_dict()

        # replacing the text "properties." from each column name
        properties = {col.replace('properties.',''):value for col,value in properties.items()}

        # Construct GeoJSON feature
        feature = {
            "type": "Feature",
            "properties": properties,
            "geometry": geometry
        }
        features.append(feature)

    # Create the GeoJSON structure
    geojson_data = {
        "type": "FeatureCollection",
        "name": name,
        "crs": {
            "type": "name",
            "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" }
        },
        "features": features
    }

    # Save GeoJSON data to file
    with open(geojson_file, 'w') as f:
        json.dump(geojson_data, f, indent=4)
    print(f"Excel data has been converted to {geojson_file}")


In [7]:
path_geojson = r'D:\PhenoCrop\3_qgis\3_Extraction Polygons\3. FINAL MASKS PYTHON\24 ProteinBar Vollebekk 20 sorter 7m_sorted_ID_polygons_shrinked.geojson'
path_xlsx = r'D:\PhenoCrop\3_qgis\3_Extraction Polygons\3. FINAL MASKS PYTHON\24 PRO_BAR_VOLL Full Field ProteinBar Vollebekk 20 sorter + NAPE 7m_sorted_ID_polygons_shrinked.xlsx'

# path = r"D:\OLF PHENO\GCPs Pheno 2019-2023.geojson"

In [6]:
geojson_to_xlsx(path_geojson)


GeoJSON data has been converted to D:\PhenoCrop\3_qgis\3_Extraction Polygons\3. FINAL MASKS PYTHON\24 ProteinBar Vollebekk 20 sorter 7m_sorted_ID_polygons_shrinked.xlsx


In [8]:
xlsx_to_geojson_beautifued(path_xlsx)

Excel data has been converted to D:\PhenoCrop\3_qgis\3_Extraction Polygons\3. FINAL MASKS PYTHON\24 PRO_BAR_VOLL Full Field ProteinBar Vollebekk 20 sorter + NAPE 7m_sorted_ID_polygons_shrinked.geojson


# Import Data for testing

In [49]:
input_file = r"sowing\24 Gene2Bread.geojson"
# input_file = r"sowing\24 OatsFrontier.geojson"
# input_file = r"sowing\24 PhenoCrop Avlingsregistrering.geojson"
# input_file = r"sowing\24 Diversity oats.geojson"
# input_file = r"sowing\24 ProteinBar Vollebekk 20 sorter 7m.geojson"
# input_file = r"sowing\24 ProteinBar NAPE 7m.geojson"
# input_file = r"sowing\24 Soraas - ProteinBar.geojson"
# input_file = r"sowing\24 E166.geojson"

# Naming the input and output files
# Generate centroids
output_cenroids = modify_filename(input_file, "1_", "_centroids", output_subfolder)
# Reorder the centroids
input_centroids = output_cenroids
output_ordered_centroids = modify_filename(input_file, "2_", "_centroids_ordered", output_subfolder)
# Update the IDs
input_ordered_centroids = output_ordered_centroids
output_centroids_with_IDs = modify_filename(input_file, "3_", "_centroids_IDs", output_subfolder)
# Generate the shrinked polygons
input_centroids_with_IDs = output_centroids_with_IDs
output_extraction_polygons = modify_filename(input_file, "4_", "_polygons_shrinked", output_subfolder)

id_start=100
id_end=126
step=100

entry_step=1
column_step=100

# Set your desired width, height, and rotation angle (in degrees)
width = 9.5  # Specify width of the rectangle
height = 2.1  # Specify height of the rectangle
rotation = 12  # Specify rotation angle of the polygons in degrees
rotation_angle = -23   # Specify rotation angle of the whole grid in degrees

In [50]:
# Performing the centroid operations
polygons_to_centroids(input_file, output_cenroids) # Find the centroid of the sowing polygons
reorder_points_by_columns(input_centroids, output_ordered_centroids, rotation_grid_for_ordering)  # Reorder the centroids. Adjust rotation_angle as needed
update_ids(input_ordered_centroids, output_centroids_with_IDs, id_start, id_end, step)  # Update the IDs

# Generate the shrinked polygons
generate_shrinked_polygons(input_centroids_with_IDs,
                            output_extraction_polygons,
                            width,
                            height,
                            rotation_polygons)

Skipping field TIME: unsupported OGR type: 10
C:\NMBU\TEMP\ipykernel_5036\3266098228.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['geometry'] = gdf['geometry'].centroid


# Organising all functions

In [14]:
source_folder = r"sowing"
sowing_files = find_files_in_folder(source_folder, 'geojson')
sowing_files

['sowing\\24 Diversity oats_sorted_ID.geojson',
 'sowing\\24 E166_sorted_ID_corrected_coordinate_system.geojson',
 'sowing\\24 Gene2Bread_sorted_ID.geojson',
 'sowing\\24 OatsFrontier_sorted_ID.geojson',
 'sowing\\24 PhenoCrop Avlingsregistrering_sorted_ID.geojson',
 'sowing\\24 ProteinBar NAPE 7m_sorted_ID.geojson',
 'sowing\\24 ProteinBar Vollebekk 20 sorter 7m_sorted_ID.geojson',
 'sowing\\24 Soraas - ProteinBar_sorted_ID.geojson']

In [22]:
source_folder = r"D:\PhenoCrop\3_qgis\3_Extraction Polygons"
sowing_files = find_files_in_folder(source_folder, 'geojson')
sowing_files

['D:\\PhenoCrop\\3_qgis\\3_Extraction Polygons\\24 E166_sorted_ID_corrected_coordinate_system.geojson',
 'D:\\PhenoCrop\\3_qgis\\3_Extraction Polygons\\E166 ORG.geojson']

In [26]:
    generate_shrinked_polygons(sowing_files[1],
                                r"D:\PhenoCrop\3_qgis\3_Extraction Polygons\2_4 E166_sorted_ID_corrected_coordinate_system.geojson",
                                12,
                                5,
                                114)

Shrinked polygons saved to D:\PhenoCrop\3_qgis\3_Extraction Polygons\2_4 E166_sorted_ID_corrected_coordinate_system.geojson


C:\Users\muij\AppData\Roaming\Python\Python312\site-packages\pyogrio\raw.py:198: RuntimeWarning: Several features with id = 1 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(


In [105]:
# SKIPPED DOING ORDERING USING PYTHON. RATHER CONVERTED ALL GEOJSON TO XLSX AND ADDED THE IDS IN EXCEL AND CONVERTED BACK ALL THE FILES BACK TO GEOJSON.
# SKIPPING THE ORDERING AND UPDATING IDs process here.

# NOTES FROM BEFORE =DISCARD=
# ORDERING IS SOMEHOW REVERESED  "ProteinBar Vollebekk"
# ORDERING NOT CORRECT "ProteinBar NAPE"
# PLOTS ARE ORDERED BUT NEED TO CHECK AND CORRECT THE IDs "E166"
# PLOTS NUMBER IS REVERSER BUT THE ORDER SEEMS CORRECT "Soraas"
# DIVERSITY NEED TO FIGURE OUT A LOGIC FOR NUMBERING


source_folder_sowing_geojson = r"sowing"
output_subfolder = "extraction"

#SIMPLE UNIQUE ID SYSTEM WITH START, ENF AND COLUMN STEP
metadata_keys = ["field", "id_start", "id_end", "step", "width", "height", "rotation_polygons", "rotation_grid_for_ordering"]
conditions = {
    "PhenoCrop": ("PhenoCrop", 100, 196, 100, 9.5, 2.1, 12, -23),
    "Frontier": ("Frontier", 100, 111, 100, 9.5, 2.1, 12, -23),
    "Gene2Bread": ("Gene2Bread", 100, 126, 100, 9.5, 2.1, 12, -23),
    "Diversity": ("Diversity", 132, 100, -1, 2, 2, 12, -23), # ID End does not work due to unusual design
    "ProteinBar Vollebekk": ("ProteinBar Vollebekk", 1100, 1122, 100, 13, 2.1, 28, -46.5), 
    "ProteinBar NAPE": ("ProteinBar NAPE", 101, 120, 100, 13.25, 2, 28, -46.5),  # Dont have the sowing shape files for this field. Generated the centroids menually
    "E166": ("E166", 1, 13, 0, 13, 3, 114, -116), # The original sowing shape files not found. These are based on the manually shrinked polygons created by students in 2024.
                                                    # The ID Logic is tricky for this one
    "Soraas": ("Soraas", 101000, 101089, 1000, 9.5, 2.1, 67, -78),
}

sowing_files = find_files_in_folder(source_folder_sowing_geojson, 'geojson')
metadata_dict = create_metadata_based_on_filename(sowing_files, conditions, metadata_keys)

for file_path, variables in metadata_dict.items():
    input_file = file_path
    # Declare variables for each metadata field
    for var_name, value in variables.items():
        globals()[var_name] = value
    file_path, field, id_start, id_end, step, width, height, rotation_polygons, rotation_grid_for_ordering

    # Naming the input and output files
    # Generate centroids
    output_cenroids = modify_filename(input_file, "1_", "_centroids", output_subfolder)
    # Reorder the centroids
    output_ordered_centroids = modify_filename(input_file, "2_", "_centroids_ordered", output_subfolder)
    # Update the IDs
    output_centroids_with_IDs = modify_filename(input_file, "3_", "_centroids_IDs", output_subfolder)
    # Generate the shrinked polygons
    output_extraction_polygons = modify_filename(input_file, "4_", "_polygons_shrinked", output_subfolder)


    # Performing the centroid operations
    polygons_to_centroids(input_file, output_cenroids) # Find the centroid of the sowing polygons
    if not "_sorted_id" in file_path.lower():
        print("here", file_path.lower())
        reorder_points_by_columns(output_cenroids, output_ordered_centroids, rotation_grid_for_ordering)  # Reorder the centroids. Adjust rotation_angle as needed
        if not "diversity" in file_path.lower():
            update_ids(output_ordered_centroids, output_centroids_with_IDs, id_start, id_end, step)  # Update the IDs
        else:
            output_centroids_with_IDs = output_ordered_centroids
    else:
        output_centroids_with_IDs = output_cenroids

    # Generate the shrinked polygons
    generate_shrinked_polygons(output_centroids_with_IDs,
                                output_extraction_polygons,
                                width,
                                height,
                                rotation_polygons)

Skipping field TIME: unsupported OGR type: 10
C:\NMBU\TEMP\ipykernel_13968\2641132132.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['geometry'] = gdf['geometry'].centroid


Shrinked polygons saved to sowing\extraction\4_24 Diversity oats_sorted_ID_polygons_shrinked.geojson
Shrinked polygons saved to sowing\extraction\4_24 E166_sorted_ID_corrected_coordinate_system_polygons_shrinked.geojson


C:\NMBU\TEMP\ipykernel_13968\2641132132.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['geometry'] = gdf['geometry'].centroid
Skipping field TIME: unsupported OGR type: 10
C:\NMBU\TEMP\ipykernel_13968\2641132132.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['geometry'] = gdf['geometry'].centroid
Skipping field TIME: unsupported OGR type: 10


Shrinked polygons saved to sowing\extraction\4_24 Gene2Bread_sorted_ID_polygons_shrinked.geojson


C:\NMBU\TEMP\ipykernel_13968\2641132132.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['geometry'] = gdf['geometry'].centroid
Skipping field TIME: unsupported OGR type: 10


...

Shrinked polygons saved to sowing\extraction\4_24 ProteinBar NAPE 7m_sorted_ID_polygons_shrinked.geojson


C:\NMBU\TEMP\ipykernel_13968\2641132132.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['geometry'] = gdf['geometry'].centroid
Skipping field TIME: unsupported OGR type: 10


Shrinked polygons saved to sowing\extraction\4_24 ProteinBar Vollebekk 20 sorter 7m_sorted_ID_polygons_shrinked.geojson


C:\NMBU\TEMP\ipykernel_13968\2641132132.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['geometry'] = gdf['geometry'].centroid


Shrinked polygons saved to sowing\extraction\4_24 Soraas - ProteinBar_sorted_ID_polygons_shrinked.geojson
